In [1]:
#### preprocessing should exists in the repo
import preprocessing
import pandas as pd
from tqdm import tqdm


ModuleNotFoundError: No module named 'nltk'

In [ ]:
###These are setups that we ran and each is slightly different than the other, you can play with it as well
# version 15
# convert_abbrev_flag = False
# model_type = 'default'
# Dropout_num = 0
# learning_rate = 1e-5
# max_len = 160
# layers = [] #not including final layer
# activation = 'relu' #for the non-final layers
# remove_emoji_flag = False
# remove_URL_flag = False
# remove_html_flag = False
# remove_punct_flag = False
# epochs=3
# batch_size=16
# validation_split=0.2
# remove_rows_based_on_cv = True #remove rows based on cross val predictions from train set
# remove_row_cutoffs = (0.4,0.6) #if cutoffs for class 0 and 1
# model_name = 'bert_default_dropout_{}_shape_{}'.format(Dropout_num,'_'.join([str(i) for i in layers]))

In [ ]:
# version 16
# convert_abbrev_flag = False
# model_type = 'dropout'
# Dropout_num = 0.5
# learning_rate = 1e-5
# max_len = 160
# layers = [] #not including final layer
# activation = 'relu' #for the non-final layers
# remove_emoji_flag = False
# remove_URL_flag = False
# remove_html_flag = False
# remove_punct_flag = False
# epochs=3
# batch_size=16
# validation_split=0.2
# remove_rows_based_on_cv = True #remove rows based on cross val predictions from train set
# remove_row_cutoffs = (0.4,0.6) #if cutoffs for class 0 and 1
# model_name = 'bert_default_dropout_{}_shape_{}'.format(Dropout_num,'_'.join([str(i) for i in layers]))

In [ ]:
# version 17
# convert_abbrev_flag = False
# model_type = 'dropout'
# Dropout_num = 0.5
# learning_rate = 1e-5
# max_len = 160
# layers = [] #not including final layer
# activation = 'relu' #for the non-final layers
# remove_emoji_flag = True
# remove_URL_flag = True
# remove_html_flag = True
# remove_punct_flag = True
# epochs=3
# batch_size=16
# validation_split=0.2
# remove_rows_based_on_cv = True #remove rows based on cross val predictions from train set
# remove_row_cutoffs = (0.4,0.6) #if cutoffs for class 0 and 1
# model_name = 'bert_default_dropout_{}_shape_{}'.format(Dropout_num,'_'.join([str(i) for i in layers]))

In [ ]:
# version 18
convert_abbrev_flag = False
model_type = 'dropout'
Dropout_num = 0.5
learning_rate = 1e-5
max_len = 160
layers = [] #not including final layer
activation = 'relu' #for the non-final layers
remove_emoji_flag = True
remove_URL_flag = False
remove_html_flag = False
remove_punct_flag = False
epochs=3
batch_size=16
validation_split=0.2
remove_rows_based_on_cv = True #remove rows based on cross val predictions from train set
remove_row_cutoffs = (0.4,0.6) #if cutoffs for class 0 and 1
model_name = 'bert_default_dropout_{}_shape_{}'.format(Dropout_num,'_'.join([str(i) for i in layers]))

# Load and Preprocess

- Load CSV files containing training data
- Do cleaning if necessary

In [ ]:
train = pd.read_csv("/data/train.csv")
test = pd.read_csv("/data/test.csv")

In [ ]:


if convert_abbrev_flag:
    train["TweetText"] = train["TweetText"].apply(lambda x: preprocessing.convert_abbrev_in_text(x))
    test["TweetText"] = test["TweetText"].apply(lambda x: preprocessing.convert_abbrev_in_text(x))
    
if remove_emoji_flag:
    train["TweetText"] = train["TweetText"].apply(lambda x: preprocessing.remove_emoji(x))
    test["TweetText"] = test["TweetText"].apply(lambda x: preprocessing.remove_emoji(x))

if remove_URL_flag:
    train['TweetText'] = train['TweetText'].apply(preprocessing.remove_URL)
    test['TweetText'] = test['TweetText'].apply(preprocessing.remove_URL)
if remove_html_flag:
    train['TweetText'] = train['TweetText'].apply(preprocessing.remove_html)
    test['TweetText'] = test['TweetText'].apply(preprocessing.remove_html)
if remove_punct_flag:
    train['TweetText'] = train['TweetText'].apply(preprocessing.remove_punct)
    test['TweetText'] = test['TweetText'].apply(preprocessing.remove_punct)

# BERT Model

- Load BERT from the Tensorflow Hub
- Load CSV files containing training data
- Load tokenizer from the bert layer
- Encode the text into tokens, masks, and segment flags
- build model

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input,Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub
# !pip install sentencepiece
import sentencepiece
import tokenization

In [ ]:
def bert_encode(texts, tokenizer,max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

# Thanks to https://www.kaggle.com/xhlulu/disaster-nlp-keras-bert-using-tfhub
def build_model(bert_layer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    
    if model_type=='default':
        # Without Dropout
        for layer in layers:
            clf_output = Dense(layer, activation=activation)(clf_output)
        out = Dense(1, activation='sigmoid')(clf_output)
    elif model_type=='dropout':
        # With Dropout(Dropout_num), Dropout_num > 0
        for layer in layers:
            x = Dropout(Dropout_num)(clf_output)
            clf_output = Dense(layer, activation=activation)(x)
        x = Dropout(Dropout_num)(clf_output)
        out = Dense(1, activation='sigmoid')(x)
    elif model_type=='GlobalAveragePooling1D':
        for layer in layers:
            if Dropout_num>0:
                clf_output = Dropout(Dropout_num)(clf_output)
            clf_output = Dense(layer, activation=activation)(clf_output)
        x = tf.keras.layers.GlobalAveragePooling1D()(sequence_output)
        out = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model


In [ ]:
%%time
import time
start=time.time()
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"#/2 is the updated version. need to try with that
# module_url = "https://tfhub.dev/tensorflow/albert_en_base/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)
end = time.time()
print("Getting BERT module time :{}".format(end - start))

Getting BERT module time :279.0259644985199
CPU times: user 42.6 s, sys: 10.6 s, total: 53.3 s
Wall time: 4min 39s


In [ ]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [ ]:
train_input = bert_encode(train.TweetText.values, tokenizer, max_len=max_len)
test_input = bert_encode(test.TweetText.values, tokenizer, max_len=max_len)
train_labels = train.Label.apply(lambda x : 0 if x=="Sports" else 1).values

print(train_input[0].shape,test_input[0].shape)

(6525, 160) (2610, 160)


In [ ]:
train_input = bert_encode(train.text.values, tokenizer, max_len=max_len)
test_input = bert_encode(test.text.values, tokenizer, max_len=max_len)
train_labels = train.target.values

print(train_input[0].shape,test_input[0].shape)

In [ ]:
model = build_model(bert_layer, max_len=160)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 160)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 160)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 160)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 1024), (None 335141889   input_word_ids[0][0]             
                                                                 input_mask[0][0]             

In [ ]:

checkpoint = ModelCheckpoint('bertwithdropout2.h5', monitor='val_loss', save_best_only=True)
start=time.time()
train_history = model.fit(
    train_input, train_labels,
    validation_split=0.2,
    epochs=epochs,
    callbacks=[checkpoint],
    batch_size=batch_size,
)
end = time.time()
print("Training the stuff time :{}".format(end - start))

Epoch 1/3
327/327 [==============================] - 577s 2s/step - loss: 0.2479 - accuracy: 0.8772 - val_loss: 0.1064 - val_accuracy: 0.9556
Epoch 2/3
327/327 [==============================] - 552s 2s/step - loss: 0.0766 - accuracy: 0.9698 - val_loss: 0.0936 - val_accuracy: 0.9670
Epoch 3/3
327/327 [==============================] - 552s 2s/step - loss: 0.0185 - accuracy: 0.9929 - val_loss: 0.1267 - val_accuracy: 0.9670
Training the shit time :1829.0113744735718


In [ ]:
start=time.time()
model.load_weights('bertwithdropout2.h5')
test_pred = model.predict(test_input)
end = time.time()
print("loading the models:{}".format(end - start))

loading the models:130.72085332870483


In [ ]:
def couldbebetter(aa):
  ree=[]
  for i in aa:
    if i==0:
      ree.append("Sports")
    else:
      ree.append("Politics")
  return ree

In [ ]:
datalabel=np.array(couldbebetter(test_pred.round().astype(int)))

In [ ]:
datatweetid=test.TweetId